<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-31 15:19:05
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.15 L
Current:  1.45 C
-------------------
Today PnL: -1.06 L (-0.73%)
Current PnL: -19.89 L (-13.03%)
CY Booked + Current PnL: -7.41 L (-4.85%)
-------------------
Total profit:  2.24 L
Total loss:  -22.13 L
-------------------
Total Booked + Current PnL: 19.57 L (15.58%)
Total Booked PnL: 39.46 L (31.41%)
Curr Year Booked PnL: 12.48 L (8.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 86.65 L (59.88%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-7.28,68.0,X-LC,4.91,231894.0,21528.0,7258.0,-0.13,10.23,3.13,13.68,37.0,2.97,1.63,28.45,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.0,103.20,54.0,M-SC,1.45,87528.0,-13249.0,13339.0,-1.84,-13.15,15.24,0.09,245.0,-0.99,0.61,14.99,OX40N,NTT,DURABLES
41,ITC,452.0,-36.23,63.0,X-LC,1.23,205429.0,5291.0,15592.0,0.32,2.64,7.59,10.44,4.0,0.34,1.44,8.35,X40,NTT,FMCG
79,UNIONBANK,163.0,-6.98,63.0,M-LC,9.92,168265.0,27425.0,17062.0,3.98,19.47,10.14,31.59,66.0,1.61,1.18,51.12,XY24,NTT,BANKS
2,ABBOTINDIA,35195.0,-12.07,30.0,X-MC,2.39,86940.0,-3738.0,18649.0,-2.09,-4.12,21.45,16.44,101.0,-0.20,0.61,15.22,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-6.98,63.0,M-LC,9.92,168265.0,27425.0,17062.0,3.98,19.47,10.14,31.59,66.0,1.61,1.18,51.12,XY24,NTT,BANKS
69,STARHEALTH,761.0,20.77,60.0,H-SC,13.47,268845.0,-12278.0,145149.0,2.73,-4.37,53.99,47.26,144.0,-0.08,1.89,44.04,XY24,NTT,INSURANCE
80,UNITDSPR,1644.0,-6.24,71.0,X-LC,4.58,245891.0,11747.0,36884.0,2.51,5.02,15.00,20.77,86.0,0.32,1.73,12.28,X40N,NTT,BREWERIES
62,SAMMAANCAP,326.0,-197.73,75.0,M-SC,30.22,168840.0,19620.0,124553.0,1.75,13.15,73.77,96.62,208.0,0.16,1.18,82.72,XY25,NTT,FINANCE
7,ASIANTILES,137.0,7088.89,60.0,L-SC,8.32,78870.0,-14940.0,91560.0,1.36,-15.93,116.09,81.67,269.0,-0.16,0.55,52.73,XR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,400.00,-16.04,39.0,H-SC,3.31,209509.0,-69050.0,326499.0,-8.34,-24.79,155.84,92.42,151.0,-0.21,1.47,21.08,XY24,NTT,BANKS
35,IEX,219.00,-34.09,44.0,H-SC,16.09,188433.0,-11521.0,108745.0,-3.27,-5.76,57.71,48.63,115.0,-0.11,1.32,5.13,XR,NTT,MISC
83,VBL,671.64,-16.73,53.0,X-LC,8.02,299796.0,-16046.0,128702.0,-3.16,-5.08,42.93,35.67,5.0,-0.12,2.10,8.07,X40N,ATH,FMCG
25,GREENPANEL,537.00,212.02,43.0,M-SC,7.53,139348.0,-43730.0,122710.0,-3.14,-23.89,88.06,43.14,230.0,-0.36,0.98,31.08,XY24,NTT,MISC
53,QUESS,986.00,-47.11,43.0,X-SC,37.85,52341.0,-12665.0,163592.0,-2.87,-19.48,312.55,232.18,198.0,-0.08,0.37,2.09,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,54.0,M-SC,1.45,87528.0,-13249.0,13339.0,-1.84,-13.15,15.24,0.09,245.0,-0.99,0.61,14.99,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,47.0,H-SC,2.79,222921.0,-46746.0,83083.0,-1.12,-17.33,37.27,13.47,138.0,-0.56,1.56,13.35,XY24,NTT,PAINTS
17,CERA,9475.0,-21.24,58.0,H-SC,2.28,143405.0,-32498.0,74513.0,0.83,-18.48,51.96,23.89,149.0,-0.44,1.01,23.85,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-23.37,48.0,H-MC,6.19,103674.0,-28161.0,70384.0,-1.50,-21.36,67.89,32.03,98.0,-0.40,0.73,16.12,OX40N,NTT,IT
67,SIS,528.0,2000.52,40.0,H-SC,3.30,84762.0,-26270.0,49874.0,-1.36,-23.66,58.84,21.26,156.0,-0.53,0.59,14.52,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,58.0,M-MC,10.30,235122.0,10160.0,157485.0,-0.37,4.52,66.98,74.53,192.0,0.06,1.65,38.23,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,52.0,M-LC,5.20,206085.0,2915.0,39218.0,-0.95,1.43,19.03,20.74,55.0,0.07,1.45,8.40,XY25,NTT,FINANCE
31,HINDZINC,730.22,27.24,48.0,M-LC,11.27,207495.0,2419.0,110159.0,-0.13,1.18,53.09,54.89,52.0,0.02,1.46,25.56,X5K,ATH,METALS
46,KPIGREEN,731.05,12.48,73.0,H-SC,5.25,134114.0,8817.0,50105.0,-0.41,7.04,37.36,47.03,141.0,0.18,0.94,67.92,MH,ATH,POWER
79,UNIONBANK,163.00,-6.98,63.0,M-LC,9.92,168265.0,27425.0,17062.0,3.98,19.47,10.14,31.59,66.0,1.61,1.18,51.12,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,12.48,73.0,H-SC,5.25,134114.0,8817.0,50105.0,-0.41,7.04,37.36,47.03,141.0,0.18,0.94,67.92,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-37.89,63.0,M-SC,4.03,128800.0,14002.0,80036.0,-1.47,12.20,62.14,81.92,223.0,0.17,0.90,52.48,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130348.0,7012.0,124613.0,-0.41,5.69,95.60,106.72,119.0,0.06,0.91,30.39,AR,ATH,MISC
31,HINDZINC,730.22,27.24,48.0,M-LC,11.27,207495.0,2419.0,110159.0,-0.13,1.18,53.09,54.89,52.0,0.02,1.46,25.56,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,43.0,M-LC,5.32,149197.0,-1748.0,111197.0,-0.53,-1.16,74.53,72.51,53.0,-0.02,1.05,4.79,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-52.51,20.0,X-LC,22.16,149719.0,-126804.0,236870.0,0.27,-45.86,158.21,39.81,54.0,-0.54,1.05,5.53,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88810.0,-39860.0,85160.0,-0.19,-30.98,95.89,35.20,219.0,-0.47,0.62,1.93,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-12.07,30.0,X-MC,2.39,86940.0,-3738.0,18649.0,-2.09,-4.12,21.45,16.44,101.0,-0.20,0.61,15.22,X40,ATH,PHARMA
23,DMART,5391.45,-18.54,31.0,X-LC,9.34,99605.0,-2981.0,29792.0,-0.26,-2.91,29.91,26.13,38.0,-0.10,0.70,21.92,X40N,ATH,FMCG
44,JSWINFRA,342.00,-22.12,32.0,X-MC,5.09,195682.0,-4665.0,35164.0,-1.86,-2.33,17.97,15.23,178.0,-0.13,1.37,29.17,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-3.68,43.0,H-LC,1.20,154785.0,-31310.0,78786.0,-0.99,-16.82,50.90,25.51,15.0,-0.40,1.09,14.06,AR,ATH,ELECTRICAL
41,ITC,452.00,-36.23,63.0,X-LC,1.23,205429.0,5291.0,15592.0,0.32,2.64,7.59,10.44,4.0,0.34,1.44,8.35,X40,NTT,FMCG
52,PGHH,17907.65,-30.81,37.0,X-MC,2.00,200160.0,-660.0,68455.0,-0.51,-0.33,34.20,33.76,80.0,-0.01,1.40,4.85,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.18,46.0,X-MC,2.91,313194.0,-14797.0,121331.0,-0.99,-4.51,38.74,32.48,92.0,-0.12,2.20,6.40,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-1.22,44.0,X-MC,4.41,207375.0,15633.0,22127.0,-2.63,8.15,10.67,19.69,99.0,0.71,1.45,15.50,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88810.0,-39860.0,85160.0,-0.19,-30.98,95.89,35.20,219.0,-0.47,0.62,1.93,X40,NTT,FOOTWEAR
53,QUESS,986.00,-47.11,43.0,X-SC,37.85,52341.0,-12665.0,163592.0,-2.87,-19.48,312.55,232.18,198.0,-0.08,0.37,2.09,XY24,NTT,MISC
20,COLPAL,3767.14,-4.10,47.0,X-MC,7.51,224030.0,-39335.0,152676.0,-0.67,-14.94,68.15,43.04,84.0,-0.26,1.57,4.01,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-14.27,48.0,X-MC,5.94,180646.0,-19860.0,117022.0,-0.51,-9.91,64.78,48.45,90.0,-0.17,1.27,4.17,X40N,ATH,FINANCE
51,PAGEIND,51769.92,-27.18,43.0,X-MC,6.73,82260.0,-1250.0,21281.0,0.45,-1.50,25.87,23.98,82.0,-0.06,0.58,4.24,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.11,43.0,X-SC,37.85,52341.0,-12665.0,163592.0,-2.87,-19.48,312.55,232.18,198.0,-0.08,0.37,2.09,XY24,NTT,MISC
57,RELAXO,1176.00,-39.97,43.0,X-SC,4.67,81874.0,-63286.0,139210.0,-1.30,-43.60,170.03,52.31,136.0,-0.45,0.57,9.59,X40N,NTT,FOOTWEAR
51,PAGEIND,51769.92,-27.18,43.0,X-MC,6.73,82260.0,-1250.0,21281.0,0.45,-1.50,25.87,23.98,82.0,-0.06,0.58,4.24,X40,ATH,APPARELS
2,ABBOTINDIA,35195.00,-12.07,30.0,X-MC,2.39,86940.0,-3738.0,18649.0,-2.09,-4.12,21.45,16.44,101.0,-0.20,0.61,15.22,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.36,88810.0,-39860.0,85160.0,-0.19,-30.98,95.89,35.20,219.0,-0.47,0.62,1.93,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.04,54.0,X-LC,10.58,287330.0,-58626.0,125333.0,0.71,-16.95,43.62,19.28,1.0,-0.47,2.02,6.22,X40,ATH,IT
39,INFY,2275.00,-18.88,52.0,X-LC,5.51,315730.0,2842.0,168852.0,-0.77,0.91,53.48,54.87,3.0,0.02,2.22,9.27,X40,BTT,IT
41,ITC,452.00,-36.23,63.0,X-LC,1.23,205429.0,5291.0,15592.0,0.32,2.64,7.59,10.44,4.0,0.34,1.44,8.35,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,8.02,299796.0,-16046.0,128702.0,-3.16,-5.08,42.93,35.67,5.0,-0.12,2.10,8.07,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,50.0,H-LC,5.44,250608.0,-11011.0,130216.0,-1.10,-4.21,51.96,45.57,7.0,-0.08,1.76,6.64,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7088.89,60.0,L-SC,8.32,78870.0,-14940.0,91560.0,1.36,-15.93,116.09,81.67,269.0,-0.16,0.55,52.73,XR,NTT,CERAMICS
19,COFFEEDAY,80.00,-45.67,46.0,L-SC,28.29,79872.0,-33677.0,73730.0,-1.21,-29.66,92.31,35.27,268.0,-0.46,0.56,95.49,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,50.0,H-SC,8.74,91800.0,-6180.0,27779.0,-0.78,-6.31,30.26,22.05,152.0,-0.22,0.64,33.44,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-37.89,63.0,M-SC,4.03,128800.0,14002.0,80036.0,-1.47,12.20,62.14,81.92,223.0,0.17,0.90,52.48,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130348.0,7012.0,124613.0,-0.41,5.69,95.60,106.72,119.0,0.06,0.91,30.39,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,12.48,73.0,H-SC,5.25,134114.0,8817.0,50105.0,-0.41,7.04,37.36,47.03,141.0,0.18,0.94,67.92,MH,ATH,POWER
81,VAIBHAVGBL,521.00,74.66,71.0,H-SC,2.21,151473.0,-31302.0,142369.0,-1.99,-17.13,93.99,60.77,125.0,-0.22,1.06,35.67,XR,NTT,JEWELLERY
85,WHIRLPOOL,2270.00,-37.89,63.0,M-SC,4.03,128800.0,14002.0,80036.0,-1.47,12.20,62.14,81.92,223.0,0.17,0.90,52.48,XR,NTT,DURABLES
5,ANGELONE,3033.00,15.31,61.0,X-SC,5.70,204459.0,13453.0,44245.0,-0.78,7.04,21.64,30.21,157.0,0.30,1.43,29.39,X40N,NTT,FINANCE
62,SAMMAANCAP,326.00,-197.73,75.0,M-SC,30.22,168840.0,19620.0,124553.0,1.75,13.15,73.77,96.62,208.0,0.16,1.18,82.72,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.50
2,50,75.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.28
LC    32.60
MC    23.11
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.14
X40      16.31
XY25     12.10
X40N     11.78
XR        9.96
AR        7.84
OX40N     6.60
X200      1.80
X5K       1.46
SR        1.06
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.45
X-LC    21.80
X-MC    18.77
M-SC    12.78
X-SC     5.60
M-LC     5.14
H-LC     4.65
H-MC     2.37
M-MC     1.65
L-SC     1.45
L-LC     1.01
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.52,-4.04,38.00
IT,12.57,-17.48,80.49
FINANCE,12.22,-8.43,57.58
MISC,7.87,-15.68,74.02
ELECTRICAL,5.83,-8.98,47.61
PAINTS,5.51,-17.43,34.94
BANKS,4.07,-16.64,84.49
INSURANCE,3.86,-0.57,38.06
AUTO,2.78,-44.81,105.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3276018.0,22
XR,1236303.0,13
AR,1170500.0,9
X40,833015.0,12
X40N,649320.0,9
OX40N,544484.0,9
XY25,457307.0,8
SR,269699.0,2
X5K,110159.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3343209.0,24
M-SC,1222488.0,15
X-MC,1201105.0,14
X-LC,1033319.0,13
X-SC,550508.0,6
M-LC,277636.0,4
H-LC,277197.0,3
L-SC,254393.0,3
H-MC,249873.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1171697.0      6
           AR         835984.0      5
           XR         774568.0      7
M-SC       XY24       700628.0      6
X-MC       XY24       580225.0      4
X-LC       X40        453964.0      5
           XY24       305840.0      2
X-SC       X40N       301756.0      4
X-MC       X40        293891.0      6
H-SC       SR         269699.0      2
           OX40N      241156.0      3
H-LC       AR         209002.0      2
X-LC       X40N       195378.0      3
H-MC       XY24       179489.0      1
X-MC       XY25       174803.0      2
L-SC       XR         165290.0      2
X-SC       XY24       163592.0      1
M-MC       XY24       157485.0      1
X-MC       X40N       152186.0      2
M-SC       OX40N      143841.0      4
           XR         127952.0      2
           AR         125514.0      2
           XY25       124553.0      1
M-LC       XR         111197.0      1
           X5K        110159.0      1
L-SC       OX40N       89103.0      1
X-SC       X40         85160.0      1
X-LC       XY25        78137.0      3
H-MC       OX40N       70384.0      1
H-LC       X200        68195.0      1
L-MC       XR          57296.0      1
H-SC       MH          50105.0      1
L-LC       XY25        40596.0      1
M-LC       XY25        39218.0      1
           XY24        17062.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
